# Homework of CH11

In [1]:
import polars as pl
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from sklearn.metrics.pairwise import cosine_similarity

將 mlxtend 函式進行封裝，使其可以直接支援 Polars 的 DataFrame。

In [2]:
def fpgrowth_pl(data: pl.DataFrame, min_support=0.5, use_colnames=False, *args, **kwargs):
    frequent_itemsets = fpgrowth(data.to_pandas(), min_support=min_support, use_colnames=use_colnames, *args, **kwargs)

    frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: list(x))
    return pl.DataFrame(frequent_itemsets)

def association_rules_pl(data: pl.DataFrame, metric="confidence", min_threshold=0.8, *args, **kwargs):
    rules = association_rules(data.to_pandas(), metric=metric, min_threshold=min_threshold, *args, **kwargs)
    rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x))
    rules["consequents"] = rules["consequents"].apply(lambda x: list(x))

    return pl.DataFrame(rules)

讀取資料。

In [3]:
df = pl.read_csv("Groceries_dataset.csv")
df

Member_number,Date,itemDescription,Quality
i64,str,str,i64
1808,"""21-07-2015""","""tropical fruit""",1
2552,"""05-01-2015""","""whole milk""",1
2300,"""19-09-2015""","""pip fruit""",1
1187,"""12-12-2015""","""other vegetables""",1
3037,"""01-02-2015""","""whole milk""",1
…,…,…,…
4471,"""08-10-2014""","""sliced cheese""",1
2022,"""23-02-2014""","""candy""",1
1097,"""16-04-2014""","""cake bar""",1


In [4]:
df.describe()

statistic,Member_number,Date,itemDescription,Quality
str,f64,str,str,f64
"""count""",38765.0,"""38765""","""38765""",38765.0
"""null_count""",0.0,"""0""","""0""",0.0
"""mean""",3003.641868,null,null,1.0
"""std""",1153.611031,null,null,0.0
"""min""",1000.0,"""01-01-2014""","""Instant food products""",1.0
"""25%""",2002.0,null,null,1.0
"""50%""",3005.0,null,null,1.0
"""75%""",4007.0,null,null,1.0
"""max""",5000.0,"""31-10-2015""","""zwieback""",1.0


看起來不需要處理空值問題。不過 `itemDescription` 欄位的資料型態是 `str`，需要轉換成 `list`。

In [5]:
data_df = df.group_by("Member_number").agg(pl.col("itemDescription"))
data_df

Member_number,itemDescription
i64,list[str]
4067,"[""domestic eggs"", ""tropical fruit"", … ""specialty chocolate""]"
3168,"[""other vegetables"", ""root vegetables"", … ""pudding powder""]"
2900,"[""pastry"", ""bottled water"", … ""canned beer""]"
3162,"[""soda"", ""bottled water""]"
1197,"[""citrus fruit"", ""root vegetables"", … ""whipped/sour cream""]"
…,…
4701,"[""soda"", ""soda"", … ""dental care""]"
3668,"[""root vegetables"", ""whole milk"", … ""rolls/buns""]"
3790,"[""pastry"", ""sausage"", … ""sweet spreads""]"


## Question 1

請使用 `Groceries_dataset` 資料集，並計算銷售資料 `support>=0.1`, `lift>=1` 以上的產品組合

- 提示: 先算出 `support>=0.1` 的 `frequent_itemsets`；再篩選 `lift>=1` 的產品組合

In [6]:
te = TransactionEncoder()
te_ary=te.fit_transform(X=data_df['itemDescription'])

data_items_dumm = pl.DataFrame(te_ary, schema=te.columns_)
data_items_dumm

Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,…,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,…,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,…,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,true,false,false
false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,…,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,true,true,false,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,…,true,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false
false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,true,false,false,false,false,false,false,…,true,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,true,false,false,false,true,true,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,f

In [7]:
frequent_itemsets = fpgrowth_pl(data_items_dumm, min_support=0.1, use_colnames=True)
frequent_itemsets

support,itemsets
f64,list[str]
0.458184,"[""whole milk""]"
0.23371,"[""tropical fruit""]"
0.177527,"[""pastry""]"
0.133145,"[""domestic eggs""]"
0.376603,"[""other vegetables""]"
…,…
0.15059,"[""whole milk"", ""yogurt""]"
0.120318,"[""yogurt"", ""other vegetables""]"
0.111339,"[""yogurt"", ""rolls/buns""]"


In [8]:
association_rules_df = association_rules_pl(frequent_itemsets, metric='lift', min_threshold=1)
association_rules_df

antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
list[str],list[str],f64,f64,f64,f64,f64,f64,f64,f64
"[""whole milk""]","[""tropical fruit""]",0.458184,0.23371,0.11647,0.254199,1.087672,0.009388,1.027473,0.148768
"[""tropical fruit""]","[""whole milk""]",0.23371,0.458184,0.11647,0.498353,1.087672,0.009388,1.080076,0.105189
"[""whole milk""]","[""other vegetables""]",0.458184,0.376603,0.19138,0.417693,1.109106,0.018827,1.070564,0.181562
"[""other vegetables""]","[""whole milk""]",0.376603,0.458184,0.19138,0.508174,1.109106,0.018827,1.101643,0.157802
"[""other vegetables""]","[""soda""]",0.376603,0.313494,0.124166,0.3297,1.051695,0.006103,1.024178,0.078849
…,…,…,…,…,…,…,…,…,…
"[""rolls/buns""]","[""yogurt""]",0.349666,0.282966,0.111339,0.318415,1.125279,0.012396,1.052011,0.171191
"[""other vegetables""]","[""rolls/buns""]",0.376603,0.349666,0.146742,0.389646,1.114335,0.015056,1.065502,0.164589
"[""rolls/buns""]","[""other vegetables""]",0.349666,0.376603,0.146742,0.419663,1.114335,0.015056,1.074197,0.157772


# Question 2

請使用 `Groceries_dataset` 資料集，並利用「使用者導向協同過濾法」，找出會員編號1000與2626這兩位相似的顧客，並進行以下事宜：

Creating `customer_item_matrix`.

In [9]:
customer_item_matrix = df.pivot(
    values="Quality",
    index="Member_number",
    on="itemDescription",
    aggregate_function="sum"
).fill_null(0)

customer_item_matrix

Member_number,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,frankfurter,chicken,butter,fruit/vegetable juice,packaged fruit/vegetables,chocolate,specialty bar,butter milk,bottled water,yogurt,sausage,brown bread,hamburger meat,root vegetables,pork,pastry,canned beer,berries,coffee,misc. beverages,ham,turkey,curd cheese,red/blush wine,frozen potato products,flour,sugar,frozen meals,…,roll products,kitchen towels,flower soil/fertilizer,cereals,meat spreads,dishes,male cosmetics,candles,whisky,tidbits,cooking chocolate,seasonal products,liqueur,abrasive cleaner,syrup,ketchup,cream,skin care,rubbing alcohol,nut snack,cocoa drinks,softener,organic products,cake bar,honey,jam,kitchen utensil,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1808,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2552,1,2,0,1,0,1,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2300,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1187,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3037,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4590,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4703,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3607,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# if number > 0, then 1; else 0
customer_item_matrix_idx = customer_item_matrix.select("Member_number")
customer_item_matrix = customer_item_matrix.with_columns(
    (pl.when(pl.col(col) > 0).then(1).otherwise(0).alias(col))
    for col in customer_item_matrix.columns
)

customer_item_matrix = customer_item_matrix.with_columns(customer_item_matrix_idx)
customer_item_matrix

Member_number,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,frankfurter,chicken,butter,fruit/vegetable juice,packaged fruit/vegetables,chocolate,specialty bar,butter milk,bottled water,yogurt,sausage,brown bread,hamburger meat,root vegetables,pork,pastry,canned beer,berries,coffee,misc. beverages,ham,turkey,curd cheese,red/blush wine,frozen potato products,flour,sugar,frozen meals,…,roll products,kitchen towels,flower soil/fertilizer,cereals,meat spreads,dishes,male cosmetics,candles,whisky,tidbits,cooking chocolate,seasonal products,liqueur,abrasive cleaner,syrup,ketchup,cream,skin care,rubbing alcohol,nut snack,cocoa drinks,softener,organic products,cake bar,honey,jam,kitchen utensil,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1808,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2552,1,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2300,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1187,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3037,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4590,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4703,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3607,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Creating User-to-User Similarity Matrix.

In [11]:
customer_item_matrix_without_member_number = customer_item_matrix.drop("Member_number")

user_user_sim_matrix = pl.DataFrame(
    cosine_similarity(customer_item_matrix_without_member_number.to_pandas()),
    schema=customer_item_matrix["Member_number"].cast(pl.String).to_list(),
)

user_user_sim_matrix = user_user_sim_matrix.with_columns(customer_item_matrix_idx)
user_user_sim_matrix

1808,2552,2300,1187,3037,4941,4501,3803,2762,4119,1340,2193,1997,4546,4736,1959,1974,2421,1513,1905,2810,2867,3962,1088,4976,4056,3611,1420,4286,4918,4783,3709,4289,1559,2900,3527,1495,…,2679,3624,1310,4083,1469,3933,2881,1084,1267,3970,3825,3000,2615,1284,1255,3991,2137,1552,2121,3777,4732,1219,4278,2296,2197,4824,4639,2456,1221,3431,3080,4590,4703,3607,4587,2417,Member_number
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
1.0,0.190693,0.1,0.129099,0.141421,0.26968,0.091287,0.223607,0.111803,0.33541,0.3,0.20702,0.074536,0.095346,0.0,0.383482,0.0,0.372678,0.105409,0.282843,0.253546,0.182574,0.182574,0.316228,0.119523,0.175412,0.0,0.095346,0.365148,0.141421,0.149071,0.158114,0.0,0.358569,0.2,0.2,0.105409,…,0.129099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.223607,0.0,0.0,0.0,0.158114,0.158114,0.0,0.0,0.158114,0.223607,0.0,0.223607,0.223607,0.0,0.0,0.0,0.223607,0.0,0.0,0.0,0.223607,0.0,0.0,0.223607,0.0,0.0,0.0,0.0,1808
0.190693,1.0,0.190693,0.246183,0.26968,0.128565,0.261116,0.355335,0.1066,0.213201,0.286039,0.197386,0.071067,0.181818,0.190693,0.365636,0.0,0.284268,0.201008,0.3371,0.161165,0.348155,0.261116,0.301511,0.113961,0.167248,0.0,0.181818,0.261116,0.0,0.284268,0.150756,0.1066,0.113961,0.190693,0.095346,0.301511,…,0.0,0.213201,0.0,0.213201,0.213201,0.150756,0.0,0.213201,0.426401,0.0,0.213201,0.0,0.150756,0.0,0.0,0.0,0.0,0.0,0.150756,0.213201,0.0,0.0,0.0,0.213201,0.0,0.0,0.213201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2552
0.1,0.190693,1.0,0.129099,0.141421,0.13484,0.182574,0.298142,0.0,0.0,0.1,0.069007,0.149071,0.190693,0.1,0.230089,0.0,0.298142,0.210819,0.212132,0.338062,0.091287,0.182574,0.105409,0.119523,0.0,0.119523,0.190693,0.182574,0.141421,0.149071,0.395285,0.223607,0.239046,0.1,0.2,0.316228,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.447214,0.0,0.0,0.0,0.223607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158114,0.223607,0.223607,0.0,0.223607,0.223607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2300
0.129099,0.246183,0.129099,1.0,0.182574,0.261116,0.353553,0.19245,0.0,0.433013,0.129099,0.267261,0.288675,0.123091,0.387298,0.19803,0.0,0.288675,0.136083,0.456435,0.218218,0.353553,0.0,0.272166,0.0,0.226455,0.154303,0.246183,0.235702,0.0,0.096225,0.204124,0.288675,0.308607,0.129099,0.129099,0.136083,…,0.166667,0.0,0.0,0.0,0.0,0.204124,0.204124,0.288675,0.0,0.0,0.288675,0.0,0.0,0.0,0.288675,0.0,0.0,0.0,0.204124,0.0,0.288675,0.0,0.204124,0.57735,0.288675,0.0,0.0,0.0,0.0,0.0,0.288675,0.288675,0.288675,0.0,0.0,0.0,1187
0.141421,0.26968,0.141421,0.182574,1.0,0.286039,0.129099,0.105409,0.158114,0.158114,0.141421,0.19518,0.210819,0.13484,0.141421,0.21693,0.0,0.210819,0.149071,0.3,0.0,0.258199,0.129099,0.298142,0.0,0.0,0.0,0.13484,0.258199,0.0,0.316228,0.111803,0.158114,0.169031,0.141421,0.0,0.149071,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316228,0.316228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316228,0.0,0.0,0.0,0.316228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3037
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.223607,0.0,0.0,0.288675,0.0,0.150756,0.204124,0.0,0.0,0.25,0.0,0.308607,0.166667,0.0,0.223607,0.0,0.0,0.166667,0.0,0.316228,0.188982,0.0,0.0,0.0,0.0,0.196116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.223607,0.0,…,0.288675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.353553,0.0,0.5,0.0,0.353553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,0.0,4590
0.0,0.0,0.0,0.288675,0.0,0.301511,0.204124,0.0,0.0,0.25,0.0,0.308607,0.166667,0.0,0.447214,0.171499,0.0,0.166667,0.235702,0.316228,0.188982,0.0,0.204124,0.235702,0.0,0.392232,0.0,0.0,0.204124,0.316228,0.166667,0.176777,0.0,0.0,0.0,0.0,0.0,…,0.57735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.353553,0.0,0.5,0.0,0.35355

## Question 2.1

列出會員編號1000顧客購買過的商品名稱。

In [12]:
product_purchased_by_1000 = (
    customer_item_matrix.filter(pl.col("Member_number") == 1000)
    .transpose(include_header=True, header_name="Product", column_names=["Purchased"])
    .filter(pl.col("Purchased") == 1)
    .to_series()
)

product_purchased_by_1000

Product
str
"""whole milk"""
"""yogurt"""
"""sausage"""
"""pastry"""
"""canned beer"""
…
"""soda"""
"""pickled vegetables"""
"""salty snack"""


## Question 2.2

列出會員編號2626顧客購買過的商品名稱。

In [13]:
product_purchased_by_2626 = (
    customer_item_matrix.filter(pl.col("Member_number") == 2626)
    .transpose(include_header=True, header_name="Product", column_names=["Purchased"])
    .filter(pl.col("Purchased") == 1)
    .to_series()
)

product_purchased_by_2626

Product
str
"""whole milk"""
"""other vegetables"""
"""chocolate"""
"""yogurt"""
"""canned beer"""
"""soda"""
"""hygiene articles"""


## Question 2.3

列出會員編號1000有買過，但2626沒有買過的商品。

In [14]:
products_id_not_purchased_by_2626 = set(product_purchased_by_1000) - set(product_purchased_by_2626)

products_id_not_purchased_by_2626

{'misc. beverages',
 'pastry',
 'pickled vegetables',
 'salty snack',
 'sausage',
 'semi-finished bread'}

# Question 3

請利用「產品導向協同過濾法」，找出通常會跟牛肉(beef)一起購買的另外五個商品名稱。

> 貼心小提醒：因為結果會包含牛肉，所以記得總共要輸出6個品項哦！

In [15]:
product_item_matrix = df.pivot(
    values="Quality",
    index="itemDescription",
    on="Member_number",
    aggregate_function="sum"
).fill_null(0)

product_item_matrix

itemDescription,1808,2552,2300,1187,3037,4941,4501,3803,2762,4119,1340,2193,1997,4546,4736,1959,1974,2421,1513,1905,2810,2867,3962,1088,4976,4056,3611,1420,4286,4918,4783,3709,4289,1559,2900,3527,…,4143,2679,3624,1310,4083,1469,3933,2881,1084,1267,3970,3825,3000,2615,1284,1255,3991,2137,1552,2121,3777,4732,1219,4278,2296,2197,4824,4639,2456,1221,3431,3080,4590,4703,3607,4587,2417
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""tropical fruit""",1,1,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,1,1,2,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""whole milk""",1,2,0,0,1,1,0,0,2,2,4,3,0,0,0,1,0,2,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,3,0,…,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""pip fruit""",0,0,1,0,0,0,0,1,0,0,0,1,2,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,…,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""other vegetables""",0,1,1,1,1,0,2,1,0,0,0,0,0,1,2,1,0,2,1,2,0,2,0,1,0,0,0,1,2,0,3,1,1,1,0,0,…,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
"""rolls/buns""",1,0,0,0,0,2,0,1,0,0,2,0,0,0,0,1,0,1,0,0,0,0,2,0,0,1,0,1,1,1,1,0,0,0,0,0,…,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""pudding powder""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""ready soups""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""make up remover""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
item_item_matrix_without_idx = product_item_matrix.drop("itemDescription")
item_item_matrix_idx = product_item_matrix.select("itemDescription")

item_item_sim_matrix = pl.DataFrame(
    cosine_similarity(item_item_matrix_without_idx.to_pandas()),
    schema=item_item_matrix_idx.to_series().to_list(),
)

item_item_sim_matrix = item_item_sim_matrix.with_columns(item_item_matrix_idx)
item_item_sim_matrix

tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,frankfurter,chicken,butter,fruit/vegetable juice,packaged fruit/vegetables,chocolate,specialty bar,butter milk,bottled water,yogurt,sausage,brown bread,hamburger meat,root vegetables,pork,pastry,canned beer,berries,coffee,misc. beverages,ham,turkey,curd cheese,red/blush wine,frozen potato products,flour,sugar,frozen meals,herbs,…,kitchen towels,flower soil/fertilizer,cereals,meat spreads,dishes,male cosmetics,candles,whisky,tidbits,cooking chocolate,seasonal products,liqueur,abrasive cleaner,syrup,ketchup,cream,skin care,rubbing alcohol,nut snack,cocoa drinks,softener,organic products,cake bar,honey,jam,kitchen utensil,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products,itemDescription
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
1.0,0.32854,0.20262,0.283033,0.278361,0.095345,0.211242,0.166683,0.175329,0.143607,0.164992,0.190369,0.095057,0.157173,0.146942,0.120261,0.225067,0.270946,0.237999,0.187561,0.139735,0.218369,0.185469,0.218548,0.188025,0.162357,0.182292,0.165197,0.130591,0.072572,0.05312,0.100471,0.055524,0.139789,0.156504,0.135939,0.091131,…,0.065777,0.048499,0.029934,0.04216,0.094026,0.041005,0.061404,0.009798,0.053177,0.042934,0.079089,0.018476,0.04136,0.036286,0.044092,0.016001,0.043379,0.024788,0.047269,0.041571,0.064922,0.026292,0.088244,0.007686,0.042776,0.0,0.076296,0.075223,0.048002,0.022628,0.016712,0.044506,0.021467,0.0,0.012394,0.0,"""tropical fruit"""
0.32854,1.0,0.282133,0.406113,0.39125,0.131088,0.280281,0.255809,0.25309,0.215953,0.239041,0.247774,0.111224,0.233145,0.155889,0.156897,0.319975,0.370205,0.327543,0.255188,0.204125,0.306941,0.240159,0.298541,0.298284,0.188657,0.223869,0.151417,0.200456,0.095446,0.106742,0.143828,0.094206,0.141285,0.210691,0.178479,0.138406,…,0.060581,0.026394,0.072145,0.094328,0.144558,0.05455,0.090969,0.058657,0.051449,0.042837,0.112464,0.035192,0.048234,0.042786,0.069322,0.030477,0.050588,0.020235,0.061096,0.06033,0.077731,0.028617,0.117059,0.029282,0.064665,0.015082,0.100344,0.062484,0.05515,0.024629,0.040928,0.044982,0.062308,0.0,0.020235,0.0,"""whole milk"""
0.20262,0.282133,1.0,0.259705,0.258508,0.110321,0.178416,0.159781,0.186753,0.140715,0.139101,0.149907,0.093192,0.157023,0.090624,0.072434,0.201561,0.235963,0.213795,0.172929,0.127598,0.210518,0.1748,0.21173,0.174281,0.146005,0.155343,0.115719,0.125343,0.057493,0.029134,0.111607,0.03105,0.076466,0.121376,0.10904,0.090665,…,0.024051,0.041167,0.045735,0.038967,0.072627,0.070384,0.077022,0.046575,0.042128,0.017007,0.078826,0.0,0.04915,0.071866,0.058218,0.009507,0.058913,0.029456,0.028086,0.0494,0.072006,0.031243,0.095032,0.027402,0.05648,0.032933,0.037777,0.056457,0.025352,0.02689,0.00993,0.060443,0.0,0.0,0.0,0.0,"""pip fruit"""
0.283033,0.406113,0.259705,1.0,0.364052,0.138777,0.265248,0.22363,0.246496,0.197271,0.24441,0.232319,0.083931,0.189083,0.163612,0.173453,0.307026,0.326959,0.30752,0.250693,0.186233,0.283807,0.242924,0.245067,0.25811,0.196889,0.206989,0.140212,0.16289,0.088661,0.081688,0.147003,0.100117,0.146383,0.181138,0.170983,0.112982,…,0.067435,0.041553,0.074091,0.071798,0.10965,0.039469,0.081837,0.052235,0.055123,0.047684,0.086706,0.006156,0.059061,0.06448,0.065294,0.037319,0.045425,0.041296,0.07481,0.050787,0.063453,0.058401,0.084531,0.030732,0.082348,0.0,0.078381,0.071233,0.060421,0.045237,0.011137,0.055079,0.047684,0.033036,0.008259,0.018468,"""other vegetables"""
0.278361,0.39125,0.258508,0.364052,1.0,0.112774,0.253463,0.191854,0.237969,0.212998,0.224287,0.246269,0.093249,0.209381,0.138009,0.149266,0.258892,0.318515,0.278007,0.226867,0.172099,0.279601,0.205324,0.252978,0.256885,

In [24]:
(
    item_item_sim_matrix
        .filter(pl.col("itemDescription") == "beef")
        .drop("itemDescription")
        .transpose(include_header=True, header_name="Product", column_names=["Similarity"])
        .sort("Similarity", descending=True)
        .filter(pl.col("Similarity") > 0)
        .head(6)
)

Product,Similarity
str,f64
"""beef""",1.0
"""whole milk""",0.255809
"""other vegetables""",0.22363
"""yogurt""",0.204807
"""soda""",0.199052
"""rolls/buns""",0.191854
